In [ ]:
import os
print(os.getcwd())


In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('..')

import utilities.functions as functions
import utilities.descritiva as descritiva

from utilities.functions import (
    load_data,
    check_key_uniqueness,
    merge_df,
    load_orders,
    process_orders_pandas
)
from utilities.descritiva import (
    plot_indicadores_mes_plataforma,
    gerar_stats,
    pedidos_group,
    criacao_ordens,
    conversao_imediata,
    analisar_ab_completo,
    calcular_poder_do_output

)

In [ ]:
from pathlib import Path
BASE_PATH = Path("/Users/maceli/ifood_cs/dados") 
df = pd.read_parquet(BASE_PATH / "silver" / "df_publico_orders.parquet")#.drop([ "delivery_address_district", "delivery_address_country",'customer_name','active','created_at','delivery_address_external_id'], axis=1)
df.head()

id_sample=['000009e29b12d8c5b9c9579e21384320c2df3bd935f19e12402be6b07a099361',
       '000021924bf8192f64a7cc17ce7c0bc511cba907308f80f3be7d81df1480e909',
       '000022ced2f3aa411b66ac41167b272ae528c4acdffeebb88a59af9ceeba4717',
       '000032b594523c3f8868edee4f1577b157e115cd01ab3110b378b06843dafd77',
       '0000bb10fb47a1d6b2d73754ef383950ef536c77d07212e431c3ff77f68834c9',
       '0000c21984ae00cefb5d4931bfa49483dde546413c9b40c4228220f27d7ecdf2',
       '00010814ab1e602ccbc9ffe3b64f44973491b2fd6d8ab7d19b138f09cdc040d0',
       '00010e03ea7d7c392a9b6cabcbed034e64fb54214e90bf89075af3d87e744023',
       '000116a7bc0a1da0c86ba2dddc02fcc70d35f8b900d69305d7eb13757f8a0c44',
       '0001226e5175177581c0e520732ba58a61dfa96d57be0caef75f5bc948887a8c']
df = df[df['customer_id'].isin(id_sample)]

In [ ]:
df["order_created_at"] = pd.to_datetime(df["order_created_at"], utc=True)
df['unique_order_hash'] = pd.util.hash_pandas_object(
        df[['customer_id', 'order_created_at','order_id']],
        index=False
    )
df = df.drop(columns=['order_id'], errors='ignore')
df["order_created_month"] = df["order_created_at"].dt.month
id_both_monht=df[df['order_created_month']==12]['customer_id'].unique()
df = df[df['customer_id'].isin(id_both_monht)].reset_index(drop=True)
df["day_month_year"] = df["order_created_at"].dt.strftime("%d-%m-%Y")
df["weekday"] = df["order_created_at"].dt.day_name()
df["hour"] = df["order_created_at"].dt.hour

df["day"] = df["order_created_at"].dt.day


In [ ]:
df_stats_mes = (
        df.groupby(['order_created_month','is_target'])
          .agg(
              total_clientes=('customer_id', 'nunique')))

df_stats_mes  

In [ ]:
#df=df[df['order_created_month']==1]

Criando dias da semana, dia, hora

In [ ]:
summary_mes=gerar_stats(df,['is_target','weekday'])
summary_mes.head()

Dias do mes

plt.figure(figsize=(12, 6))
sns.violinplot(data=df, x='weekday', y='order_total_amount', hue='is_target',
               order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title('Densidade e distribuição por dia da semana')
plt.xticks(rotation=45)
plt.show()

Testar ver se as variaveis influenciam no numero de pedidos

In [ ]:
#plot_indicadores_mes_plataforma(summary_mes)

In [ ]:
summary_month_day=gerar_stats(df,['is_target','day'])
summary_month_day.sort_values("day").head()

In [ ]:
summary_hora=gerar_stats(df,['is_target','hour'])
summary_hora.sort_values("hour").head()

In [ ]:
df_analise_completa = (
        df.groupby(['order_created_month', 'customer_id'])
        .agg(
            total_pedidos=('unique_order_hash', 'count'),
            total_amount=('order_total_amount', 'sum')
        )
        .reset_index()
        .assign(
            categoria_pedidos=lambda x: x['total_pedidos']
                .apply(lambda y: '1_pedido' if y == 1 else '2+_pedidos')
        )
        .groupby(['order_created_month', 'total_pedidos'])
        .agg(
            total_clientes=('customer_id', 'nunique'),
            #total_pedidos=('total_pedidos', 'sum'),
            total_amount=('total_amount', 'sum'),
            #avg_amount_por_cliente=('total_amount', 'mean')
       )
        .reset_index()
    )
df_analise_completa

In [ ]:
summary_state=gerar_stats(df,['is_target','delivery_address_state'])
summary_state.sort_values("perc_clientes").head()

# Gráfico de barras com linhas - Estados no eixo X
fig, axes = plt.subplots(2, 2, figsize=(20, 12))
fig.suptitle('% Pedidos (Barras) vs % Valor e % Ordem (Linhas) por Estado', 
             fontsize=16, fontweight='bold')

months = [12, 1]
month_names = {12: 'Dezembro', 1: 'Janeiro'}
targets = ['target', 'control']
target_names = {'target': 'Target', 'control': 'Control'}

for i, month in enumerate(months):
    for j, target_val in enumerate(targets):
        ax = axes[i, j]
        
        # Filtrar dados para o mês e target específico
        filtered_data = summary_state[
            (summary_state['order_created_month'] == month) & 
            (summary_state['is_target'] == target_val)
        ]
        
        if len(filtered_data) > 0:
            # Agrupar por estado e ordenar por % pedidos
            estado_data = filtered_data.groupby('delivery_address_state').first()
            estado_data = estado_data.sort_values('perc_pedidos', ascending=False)
            
            # Pegar top 10 estados para não poluir o gráfico
            top_estados = estado_data.head(10)
            estados_ordenados = top_estados.index.tolist()
            
            # Criar segundo eixo Y para as linhas
            ax2 = ax.twinx()
            
            # Barras para % pedidos
            bars = ax.bar(estados_ordenados, top_estados['perc_pedidos'], 
                         alpha=0.7, color='lightblue', label='% Pedidos', edgecolor='navy')
            
            # Linhas para % valor e % ordem
            line1 = ax2.plot(estados_ordenados, top_estados['perc_amount'], 
                            'ro-', linewidth=3, markersize=8, label='% Valor')
            line2 = ax2.plot(estados_ordenados, top_estados['total_ordem'], 
                            'go-', linewidth=3, markersize=8, label='% Ordem')
            
            # Adicionar valores nas barras
            for bar, estado in zip(bars, estados_ordenados):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                       f'{height:.1f}%', ha='center', va='bottom', 
                       fontweight='bold', fontsize=9)
            
            # Adicionar valores nas linhas
            for estado, valor_amount, valor_ordem in zip(estados_ordenados, 
                                                        top_estados['perc_amount'], 
                                                        top_estados['total_ordem']):
                ax2.text(estado, valor_amount, f'{valor_amount:.1f}%', 
                        ha='center', va='bottom', fontweight='bold', 
                        color='red', fontsize=9)
                ax2.text(estado, valor_ordem, f'{valor_ordem:.1f}%', 
                        ha='center', va='top', fontweight='bold', 
                        color='green', fontsize=9)
            
            # Configurações do primeiro eixo (barras)
            ax.set_xlabel('Estado', fontsize=12)
            ax.set_ylabel('% Pedidos', color='navy', fontsize=12)
            ax.tick_params(axis='y', labelcolor='navy')
            ax.tick_params(axis='x', rotation=45)
            
            # Configurações do segundo eixo (linhas)
            ax2.set_ylabel('% Valor e % Ordem', color='red', fontsize=12)
            ax2.tick_params(axis='y', labelcolor='red')
            
            # Combinar legendas
            lines1, labels1 = ax.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
            
            ax.set_title(f'{month_names[month]} - {target_names[target_val]}', 
                        fontsize=14, fontweight='bold')
            ax.grid(True, alpha=0.3, axis='y')
        
        else:
            ax.text(0.5, 0.5, 'Sem dados', ha='center', va='center', 
                   transform=ax.transAxes, fontsize=12, style='italic')

plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()

In [ ]:
df.head()

Numero de pedidos por cliente exatamente um ou mais de um

In [ ]:
df.columns

In [ ]:
pedidos_group(df,group_vars='is_target')

In [ ]:
df, dias_0=criacao_ordens(df,group_vars=['is_target','order_created_month'])
df
#print('Media da diferenca em dias entre a primeira e segunda ordem, nao levando em consideracao o periodo do mes',dias_0)

Dias entre a primeira ordem e segunda

In [ ]:
summary_mes = gerar_stats(
    df[(df['rank_month'] == 2)&(df['order_created_month']==12)],
    ['days_since_first_order_month','is_target']
)


In [24]:
summary_mes[summary_mes['is_target']=='target'].head(10)

,order_created_month,days_since_first_order_month,is_target,total_clientes,total_pedidos,total_ordem,perc_clientes,perc_pedidos,perc_amount
0,12,0,target,23415,23415,1049863.45,9.22,9.22,8.58
1,12,1,target,19015,19015,884087.51,7.49,7.49,7.23
3,12,2,target,13759,13759,651399.58,5.42,5.42,5.32
5,12,3,target,11190,11190,538865.25,4.41,4.41,4.40
6,12,4,target,9731,9731,474980.99,3.83,3.83,3.88
7,12,6,target,9627,9627,472435.25,3.79,3.79,3.86
9,12,7,target,9058,9058,448091.84,3.57,3.57,3.66
10,12,5,target,8951,8951,435555.79,3.52,3.52,3.56
14,12,8,target,6296,6296,305304.39,2.48,2.48,2.50
17,12,9,target,4882,4882,234990.60,1.92,1.92,1.92


In [25]:

summary_mes[summary_mes['is_target']=='control'].head(10)

,order_created_month,days_since_first_order_month,is_target,total_clientes,total_pedidos,total_ordem,perc_clientes,perc_pedidos,perc_amount
2,12,0,control,15734,15734,858043.23,6.20,6.20,7.01
4,12,1,control,12684,12684,590519.89,4.99,4.99,4.83
8,12,2,control,9295,9295,439651.14,3.66,3.66,3.59
11,12,3,control,7482,7482,362722.87,2.95,2.95,2.96
12,12,6,control,6493,6493,319483.55,2.56,2.56,2.61
13,12,4,control,6329,6329,306025.36,2.49,2.49,2.50
15,12,5,control,6077,6077,294220.82,2.39,2.39,2.40
16,12,7,control,6039,6039,293048.15,2.38,2.38,2.40
19,12,8,control,4202,4202,197916.12,1.65,1.65,1.62
24,12,9,control,3329,3329,160387.83,1.31,1.31,1.31


"""
ANÁLISE DO TEMPO PARA SEGUNDA COMPRA
====================================

PRINCIPAIS DESCOBERTAS:
----------------------
• 15.44% dos clientes fazem a 2ª compra NO MESMO DIA
• 37% das segundas compras ocorrem nas primeiras 48h
• 68% do potencial é convertido na primeira semana
• Queda de 52% no volume entre dia 0 e dia 3

INSIGHTS ESTRATÉGICOS:
---------------------
1. Janela crítica: primeiras 48h (37% das conversões)
2. Segmentar por velocidade: 
   - Imediatos (dia 0): 15.44%
   - Rápidos (dias 1-3): 28.78% 
   - Tardios (dias 4-9): 45.89%
3. Campanhas escalonadas por tempo
4. Foco em retenção imediata pós-primeira compra

PRÓXIMOS PASSOS:
---------------
• Analisar perfil dos compradores imediatos
• Criar campanhas por janela temporal
• Testar incentivos para acelerar segunda compra
• Monitorar ticket médio por cohort
"""

In [ ]:
def conversao_imediata(
    df,
    mes_base=12,
    r_ordem=2,
    window_start=1,  # NOVO: início da janela
    window_end=1,    # NOVO: fim da janela  
    fill_value=-999
):
    """
    Versão atualizada que aceita janela range (start, end)
    """
    # 1) Filtra pedidos do mês
    df_publico = df[df['order_created_month'] == mes_base].copy()

    # 2) Clientes que fizeram r_ordem-ésimo pedido no mês
    df_d = df_publico[df_publico['rank_month'] == r_ordem].drop_duplicates().copy()

    # 3) Flag de conversão: dentro da janela range em dias
    df_d['converteu'f"{window_start}-{window_end}d"] = (df_d['days_since_first_order_month'] >= window_start) & (df_d['days_since_first_order_month'] <= window_end)

    # 4) Mantém só o que precisamos da 2ª compra
    df_d = df_d[['customer_id', 'converteu'f"{window_start}-{window_end}d", 'days_since_first_order_month', 'order_total_amount']]

    # 5) Base de todos os clientes ativos no mês (um registro por customer_id x is_target)
    df_publico = df_publico[['customer_id', 'is_target']].drop_duplicates().merge(
        df_d,
        on='customer_id',
        how='left'
    )

    # 6) Tratar tipos / NaNs
    df_publico['converteu'f"{window_start}-{window_end}d"] = df_publico['converteu'f"{window_start}-{window_end}d"].fillna(False).astype(int)
    df_publico['order_total_amount'] = df_publico['order_total_amount'].fillna(0).astype(float)

    # Se quiser -999 onde ainda sobrou NaN (ex: days_since_first_order_month)
    df_publico = df_publico.fillna(fill_value)
    df_publico['windows']=f"{window_start}-{window_end}d"
    # 8) Resumo por grupo
    resumo = (
        df_publico.groupby("is_target", as_index=False)
                  .agg(
                      total_clientes=('customer_id', 'nunique'),
                      clientes_convertidos=('converteu'f"{window_start}-{window_end}d", 'sum'),
                      total_amount=('order_total_amount', 'sum'),
                     total_amount_convertido=('order_total_amount',lambda x: x[df_publico.loc[x.index, 'converteu'f"{window_start}-{window_end}d"] == 1].sum())
                  ).assign(
                taxa_conversao=lambda x: (x['clientes_convertidos'] / x['total_clientes'] * 100).round(2),
                window_range=f"{window_start}-{window_end}d",

                mes_base=mes_base
            ).reset_index(drop=True)
    )

    return resumo, df_publico

In [ ]:
janelas = [(0, 1), (0, 3), (0, 4), (10, 16)]

df_base_final = None
todos_resumos = []

for window_start, window_end in janelas:

    resumo, df_p = conversao_imediata(
        df,
        mes_base=12,
        r_ordem=2,
        window_start=window_start,
        window_end=window_end,
        fill_value=-999
    )

    # guardar resumo de cada iteração
    todos_resumos.append(resumo)

    # nome da coluna que a função CRIA
    col_origem = f"converteu{window_start}-{window_end}d"
    # nome da coluna que você quer na base final
    col_destino = f"converteu_{window_start}_{window_end}"

    # cria a nova coluna usando a coluna dinâmica da função
    df_p[col_destino] = df_p[col_origem]

    # montar/atualizar base final mantendo target e valor
    if df_base_final is None:
        df_base_final = df_p[
            ['customer_id', 'is_target', 'order_total_amount', col_destino]
        ].copy()
    else:
        df_base_final = df_base_final.merge(
            df_p[['customer_id', col_destino]],
            on='customer_id',
            how='left'
        )

# preencher NaN das flags com 0
for window_start, window_end in janelas:
    col_destino = f"converteu_{window_start}_{window_end}"
    df_base_final[col_destino] = df_base_final[col_destino].fillna(0).astype(int)

# resumo com TODAS as iterações
df_resumo_final = pd.concat(todos_resumos, ignore_index=True)


In [ ]:
df_base_final

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest


def teste_proporcao_por_janela(df):
    resultados = []

    # colunas de conversão (flags)
    colunas_flags = [c for c in df.columns if c.startswith("converteu_")]

    for col in colunas_flags:

        # separar grupos
        control = df[df['is_target'] == 'control']
        target  = df[df['is_target'] == 'target']

        # conversões
        conv_control = control[col].sum()
        conv_target  = target[col].sum()

        # totais
        total_control = control['customer_id'].nunique()
        total_target  = target['customer_id'].nunique()

        # teste de proporção
        count = np.array([conv_target, conv_control])
        nobs  = np.array([total_target, total_control])

        z_stat, p_value = proportions_ztest(count, nobs)

        taxa_control = conv_control / total_control if total_control > 0 else 0
        taxa_target  = conv_target / total_target if total_target > 0 else 0

        resultados.append({
            'janela': col,
            'total_control': total_control,
            'conv_control': conv_control,
            'total_target': total_target,
            'conv_target': conv_target,
            'taxa_control': round(taxa_control*100, 2),
            'taxa_target': round(taxa_target*100, 2),
            'z_stat': z_stat,
            'p_value': p_value,
            'significativo_5%': p_value < 0.05
        })

    return pd.DataFrame(resultados)


In [ ]:
teste_proporcao_por_janela(df_base_final)

In [ ]:
def analise_completa_ranges_personalizados(df, ranges_janelas, mes_base=12, r_ordem=2, alpha=0.05):
    """

    """
    import pandas as pd
    import numpy as np
    from scipy.stats import mannwhitneyu, ttest_ind, t
    from scipy import stats
    from statsmodels.stats.proportion import proportions_ztest, proportion_confint
    import warnings
    warnings.filterwarnings('ignore')
    
    descontos = [0.05, 0.10, 0.15, 0.20]  
    
    resultados_combinados = []

    for janela_inicio, janela_fim in ranges_janelas:
        for desconto in descontos:
            print(f"🧪 Testando {desconto*100}% de desconto na janela {janela_inicio}-{janela_fim} dias...")
            
            # Executa função de conversão
            resumo, df_publico_mes = conversao_imediata(
                df,
                mes_base=mes_base,
                r_ordem=r_ordem,
                desconto_p=desconto,
                window_start=janela_inicio,
                window_end=janela_fim,
                fill_value=-999
            )
            
            # 🔥 ANÁLISE DE PROPORÇÃO (CONVERSÃO)
            try:
                total_target = resumo.loc[resumo['is_target'] == 'target', 'total_clientes'].values[0]
                total_control = resumo.loc[resumo['is_target'] == 'control', 'total_clientes'].values[0]

                conversoes_target = resumo.loc[resumo['is_target'] == 'target', 'clientes_convertidos'].values[0]
                conversoes_control = resumo.loc[resumo['is_target'] == 'control', 'clientes_convertidos'].values[0]
                
                print(f"   → Target: {conversoes_target}/{total_target} conversões")
                print(f"   → Control: {conversoes_control}/{total_control} conversões")
                print(f"   → Amount Target: {resumo.loc[resumo['is_target'] == 'target', 'total_amount'].values[0]:.2f}")
                print(f"   → Amount Control: {resumo.loc[resumo['is_target'] == 'control', 'total_amount'].values[0]:.2f}")
                
                if total_target > 0 and total_control > 0 and (conversoes_target + conversoes_control) > 0:
                    count = np.array([conversoes_target, conversoes_control])
                    nobs = np.array([total_target, total_control])

                    # TESTE Z DE PROPORÇÃO
                    z_stat_prop, p_value_prop = proportions_ztest(count, nobs)

                    taxa_target = conversoes_target / total_target
                    taxa_control = conversoes_control / total_control
                    diferenca_absoluta = taxa_target - taxa_control
                    lift_relativo = diferenca_absoluta / taxa_control if taxa_control > 0 else 0

                    # INTERVALOS DE CONFIANÇA
                    ci_target = proportion_confint(conversoes_target, total_target, alpha=alpha)
                    ci_control = proportion_confint(conversoes_control, total_control, alpha=alpha)

                    # Adiciona métricas ao resumo
                    resumo['z_stat_proporcao'] = z_stat_prop
                    resumo['p_value_proporcao'] = p_value_prop
                    resumo['significativo_proporcao'] = p_value_prop < alpha
                    resumo['lift_relativo'] = lift_relativo
                    resumo['diferenca_absoluta'] = diferenca_absoluta
                    resumo['ic_target_inf'] = ci_target[0]
                    resumo['ic_target_sup'] = ci_target[1]
                    resumo['ic_control_inf'] = ci_control[0]
                    resumo['ic_control_sup'] = ci_control[1]
                    
            except Exception as e:
                print(f"⚠️ Erro no teste de proporção: {e}")
                resumo['z_stat_proporcao'] = np.nan
                resumo['p_value_proporcao'] = np.nan
                resumo['significativo_proporcao'] = False
            
          
            try:
            
                valores_control = df_publico_mes[
                    (df_publico_mes['is_target'] == 'control') & 
                    (df_publico_mes['converteu'] == 1) &
                    (df_publico_mes['order_total_amount'] >= 0) &
                    # ✅ FILTRO CORRETO pela coluna temporal real
                    (df_publico_mes['days_since_first_order_month'] >= janela_inicio) &
                    (df_publico_mes['days_since_first_order_month'] <= janela_fim)
                ]['order_total_amount']
                
                valores_target = df_publico_mes[
                    (df_publico_mes['is_target'] == 'target') & 
                    (df_publico_mes['converteu'] == 1) &
                    (df_publico_mes['order_total_amount'] >= 0) &
                    # ✅ FILTRO CORRETO pela coluna temporal real
                    (df_publico_mes['days_since_first_order_month'] >= janela_inicio) &
                    (df_publico_mes['days_since_first_order_month'] <= janela_fim)
                ]['order_total_amount']
                
                print(f"   → Transações Target (dias {janela_inicio}-{janela_fim}): {len(valores_target)}")
                print(f"   → Transações Control (dias {janela_inicio}-{janela_fim}): {len(valores_control)}")
                
                if len(valores_target) > 0:
                    print(f"   → Soma REAL Target: R$ {valores_target.sum():.2f}")
                if len(valores_control) > 0:
                    print(f"   → Soma REAL Control: R$ {valores_control.sum():.2f}")
                
                if len(valores_control) > 0 and len(valores_target) > 0:
                    # TESTES ESTATÍSTICOS
                    stat_mw, p_value_mw = mannwhitneyu(valores_target, valores_control, alternative='two-sided')
                    t_stat_w, p_value_w = ttest_ind(valores_target, valores_control, equal_var=False)
                    t_stat_n, p_value_n = ttest_ind(valores_target, valores_control)
                    
                    # MÉTRICAS
                    media_control = valores_control.mean()
                    media_target = valores_target.mean()
                    dp_control = valores_control.std()
                    dp_target = valores_target.std()
                    n_control = len(valores_control)
                    n_target = len(valores_target)
                    
                    # Cohen's d
                    dp_pooled = np.sqrt(((n_control-1)*dp_control**2 + (n_target-1)*dp_target**2) / (n_control + n_target - 2))
                    cohen_d = (media_target - media_control) / dp_pooled
                    
                    # IC 95% da diferença de médias
                    ep_diferenca = np.sqrt(dp_control**2/n_control + dp_target**2/n_target)
                    t_critico = stats.t.ppf(0.975, df=n_control+n_target-2)
                    diferenca = media_target - media_control
                    ic_inferior = diferenca - t_critico * ep_diferenca
                    ic_superior = diferenca + t_critico * ep_diferenca
                    
                    # Adiciona ao resumo
                    resumo['mannwhitney_stat'] = stat_mw
                    resumo['mannwhitney_pvalue'] = p_value_mw
                    resumo['ttest_welch_stat'] = t_stat_w
                    resumo['ttest_welch_pvalue'] = p_value_w
                    resumo['ttest_normal_stat'] = t_stat_n
                    resumo['ttest_normal_pvalue'] = p_value_n
                    resumo['cohen_d'] = cohen_d
                    resumo['diferenca_medias'] = diferenca
                    resumo['ic_medias_inferior'] = ic_inferior
                    resumo['ic_medias_superior'] = ic_superior
                    resumo['media_control'] = media_control
                    resumo['media_target'] = media_target
                    resumo['significativo_valores'] = p_value_w < alpha
                    resumo['n_transacoes_target'] = n_target
                    resumo['n_transacoes_control'] = n_control
                    resumo['soma_real_target'] = valores_target.sum()
                    resumo['soma_real_control'] = valores_control.sum()
                    
                    print(f"   → Média Target: R$ {media_target:.2f}, Control: R$ {media_control:.2f}")
                    print(f"   → P-value Welch: {p_value_w:.6f}")
                    
            except Exception as e:
                print(f"⚠️ Erro na análise de valores: {e}")
                resumo['mannwhitney_pvalue'] = np.nan
                resumo['ttest_welch_pvalue'] = np.nan
                resumo['ttest_normal_pvalue'] = np.nan
                resumo['cohen_d'] = np.nan
                resumo['significativo_valores'] = False
            
            # Adiciona informações da combinação
            df_resumo = resumo.copy()
            df_resumo['janela_inicio'] = janela_inicio
            df_resumo['janela_fim'] = janela_fim
            df_resumo['janela_range'] = f"{janela_inicio}-{janela_fim}d"
            df_resumo['desconto_percentual'] = desconto * 100
            df_resumo['desconto_decimal'] = desconto
            resultados_combinados.append(df_resumo)
    
    # Combina todos os resultados
    df_resultados_completo = pd.concat(resultados_combinados, ignore_index=True)
    
    # 🔍 VERIFICAÇÃO FINAL - Identificar se o problema persiste
    print("\n🔍 VERIFICAÇÃO FINAL - COMPARAÇÃO ENTRE JANELAS:")
    
    problemas_encontrados = []
    
    for janela in ranges_janelas:
        janela_str = f"{janela[0]}-{janela[1]}d"
        dados_janela = df_resultados_completo[df_resultados_completo['janela_range'] == janela_str]
        
        # Verificar se os amounts são diferentes entre janelas
        amount_target = dados_janela[dados_janela['is_target'] == 'target']['total_amount'].unique()
        amount_control = dados_janela[dados_janela['is_target'] == 'control']['total_amount'].unique()
        
        print(f"\n📊 Janela {janela_str}:")
        print(f"   Target: R$ {amount_target[0]:.2f} | Control: R$ {amount_control[0]:.2f}")
        
        # Verificar soma real vs total_amount do resumo
        soma_real_target = dados_janela[dados_janela['is_target'] == 'target']['soma_real_target'].unique()
        soma_real_control = dados_janela[dados_janela['is_target'] == 'control']['soma_real_control'].unique()
        
        if len(soma_real_target) > 0 and not np.isnan(soma_real_target[0]):
            print(f"   Soma REAL Target: R$ {soma_real_target[0]:.2f}")
            if abs(soma_real_target[0] - amount_target[0]) > 1.0:
                problemas_encontrados.append(f"INCONSISTÊNCIA: Janela {janela_str} - Target: Resumo R$ {amount_target[0]:.2f} vs Real R$ {soma_real_target[0]:.2f}")
    
    if problemas_encontrados:
        print("\n🚨 PROBLEMAS IDENTIFICADOS:")
        for problema in problemas_encontrados:
            print(f"   {problema}")
        print("\n💡 SOLUÇÃO: O problema está na função conversao_imediata() - ela não está filtrando corretamente por janela temporal")
    else:
        print("\n✅ Todas as janelas têm valores consistentes!")
    
    # Salva resultados
    with pd.ExcelWriter('analise_completa_CORRIGIDA_FINAL.xlsx', engine='openpyxl') as writer:
        df_resultados_completo.to_excel(writer, sheet_name='Resultados_Completos', index=False)
        
        # Cria resumo para análise
        resumo_analise = df_resultados_completo.groupby(['janela_range', 'desconto_percentual', 'is_target']).agg({
            'total_amount': 'first',
            'soma_real_target': 'first',
            'soma_real_control': 'first',
            'n_transacoes_target': 'first',
            'n_transacoes_control': 'first',
            'taxa_conversao': 'first',
            'p_value_proporcao': 'first',
            'ttest_welch_pvalue': 'first'
        }).reset_index()
        resumo_analise.to_excel(writer, sheet_name='Resumo_Analise', index=False)
    
    print("✅ Análise final concluída! Verifique o arquivo Excel gerado.")
    return df_resultados_completo

# EXECUTAR ANÁLISE CORRIGIDA
ranges_personalizados = [
    (0, 4),    # Primeiros 4 dias
    #(5, 19),   # Dias 5-19  
    #(20, 30),  # Dias 20-30
    #(5, 6)     # Foco específico
]

df_resultados_corrigido = analise_completa_ranges_personalizados(
    df, 
    ranges_janelas=ranges_personalizados
)

In [ ]:
ranges_personalizados = [
    (0, 1),    
    (1,2),
    (0,5),
    (5, 7),   
    (15,16),
    (20, 30),  
    (5, 6)    
]

df_resultados = analise_completa_ranges_personalizados(
    df, 
    ranges_janelas=ranges_personalizados
)

A estratégia aumenta a PROPORÇÃO DE CLIENTES que retornam para uma segunda compra em ate 24 horas apos a segunda

In [ ]:
resultado = analisar_ab_completo(
    df_base=df_publico_mes,
    alpha=0.05
    
)

resultado


In [ ]:
from scipy.stats import mannwhitneyu,ttest_ind

# Filtrar apenas clientes que converteram
valores_control = df_publico_mes[(df_publico_mes['is_target'] == 'control') & (df_publico_mes['converteu'] == 1)]['order_total_amount']
valores_target = df_publico_mes[(df_publico_mes['is_target'] == 'target') & (df_publico_mes['converteu'] == 1)]['order_total_amount']

# Executar teste
stat, p_value = mannwhitneyu(valores_target, valores_control, alternative='two-sided')
print(stat, p_value)
t_stat_w, p_value_w = ttest_ind(valores_target, valores_control, equal_var=False)
print(t_stat_w, p_value_w)
t_stat_n, p_value_n = ttest_ind(valores_target, valores_control)
print(t_stat_n,p_value_n)

In [ ]:
# Suas amostras
valores_control = df_publico_mes[df_publico_mes['is_target'] == 'control']['order_total_amount']
valores_target = df_publico_mes[df_publico_mes['is_target'] == 'target']['order_total_amount']

# Estatísticas básicas
media_control = valores_control.mean()
media_target = valores_target.mean()
dp_control = valores_control.std()
dp_target = valores_target.std()
n_control = len(valores_control)
n_target = len(valores_target)

# Cohen's d
dp_pooled = np.sqrt(((n_control-1)*dp_control**2 + (n_target-1)*dp_target**2) / (n_control + n_target - 2))
cohen_d = (media_target - media_control) / dp_pooled

# IC 95% da diferença
ep_diferenca = np.sqrt(dp_control**2/n_control + dp_target**2/n_target)
t_critico = stats.t.ppf(0.975, df=n_control+n_target-2)
diferenca = media_target - media_control
ic_inferior = diferenca - t_critico * ep_diferenca
ic_superior = diferenca + t_critico * ep_diferenca

print(f"Cohen's d: {cohen_d:.6f}")
print(f"IC 95% diferença: [{ic_inferior:.2f}, {ic_superior:.2f}]")

In [ ]:
from scipy.stats import mannwhitneyu

# Filtrar apenas clientes que converteram
valores_control = df_publico_mes[(df_publico_mes['is_target'] == 'control') & (df_publico_mes['converteu'] == 1)]['amount_pos_desc']
valores_target = df_publico_mes[(df_publico_mes['is_target'] == 'target') & (df_publico_mes['converteu'] == 1)]['amount_pos_desc']

# Executar teste
stat, p_value = mannwhitneyu(valores_target, valores_control, alternative='two-sided')
stat, p_value

O teste com valor pós-desconto é significativo porque está medindo o próprio desconto, não o comportamento do cliente"

In [ ]:
print(df_publico_mes[df_publico_mes['is_target'] == 'target']['order_total_amount'].describe())
print(df_publico_mes[df_publico_mes['is_target'] == 'target']['amount_pos_desc'].describe())
print(df_publico_mes[df_publico_mes['is_target'] == 'control']['amount_pos_desc'].describe())

"""
📈 TESTE DE VALORES - MANN-WHITNEY
==================================

RESULTADOS:
----------
• Estatística U: 9.269.144,5
• p-value: 0,658

CONCLUSÃO:
---------
✅ RESULTADO VÁLIDO: Não há diferença significativa 
   nos valores das segundas compras entre grupos

CONSISTÊNCIA:
------------
• Estratégia AUMENTA quantidade de clientes que voltam
• Estratégia NÃO ALTERA valor gasto pelos clientes
"""

"""
📊 RESUMO - TESTE A/B SEGUNDA COMPRA
====================================

✅ RESULTADOS POSITIVOS:
-----------------------
• Aumento de 13.56% na taxa de retorno de clientes
• +1.653 clientes adicionais no grupo target
• Resultado estatisticamente significativo (p-value < 0.0001)
• ROI de 11.24x - cada R$1 em desconto gerou R$11.24 em vendas extras

➡️ RESULTADOS NEUTROS:
----------------------
• Valor médio das segundas compras não mudou
• Ticket médio estável entre grupos

💰 IMPACTO FINANCEIRO:
----------------------
• Incremento líquido: R$ 266.628 após custos de desconto
• Estratégia é financeiramente viável

🚀 RECOMENDAÇÕES:
----------------
1. Manter estratégia de incentivos para segunda compra
2. Desenvolver táticas de upsell para aumentar ticket médio
3. Investigar perfil de produtos comprados por clientes novos
4. Expandir teste para outras bases

PRÓXIMOS PASSOS:
---------------
• Campanhas de aumento de ticket médio
• Análise de perfil por categoria de produto
• Teste de diferentes valores de desconto
"""

In [ ]:
from statsmodels.stats.power import TTestIndPower

# Pegue os dados automaticamente
n_target = len(df_publico_mes[df_publico_mes['is_target'] == 'target'])
n_control = len(df_publico_mes[df_publico_mes['is_target'] == 'control'])
std = df_publico_mes['order_total_amount'].std()

# Calcule o effect size da diferença REAL que você encontrou
media_target = df_publico_mes[df_publico_mes['is_target'] == 'target']['order_total_amount'].mean()
media_control = df_publico_mes[df_publico_mes['is_target'] == 'control']['order_total_amount'].mean()
diferenca_real = media_target - media_control
effect_size_real = diferenca_real / std

# Calcule o PODER REAL
analysis = TTestIndPower()
poder_real = analysis.solve_power(
    effect_size=effect_size_real,  # usa a diferença REAL
    nobs1=n_target,
    alpha=0.05,
    ratio=n_control/n_target
)

print(f"🎯 PODER REAL DO SEU TESTE:")
print(f"• Diferença encontrada: R$ {diferenca_real:.2f}")
print(f"• Effect Size: {effect_size_real:.4f}")
print(f"• Poder estatístico: {poder_real:.1%}")

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
from statsmodels.stats.power import TTestIndPower

def calcular_mde_completo(df):
    # Pega dados básicos
    n_target = len(df[df['is_target'] == 'target'])
    n_control = len(df[df['is_target'] == 'control'])
    std = df['order_total_amount'].std()
    
    # Calcula effect size real
    media_target = df[df['is_target'] == 'target']['order_total_amount'].mean()
    media_control = df[df['is_target'] == 'control']['order_total_amount'].mean()
    effect_size_real = (media_target - media_control) / std
    
    # Calcula poder atual
    analysis = TTestIndPower()
    poder_atual = analysis.solve_power(
        effect_size=effect_size_real,
        nobs1=n_target,
        alpha=0.05,
        ratio=n_control/n_target
    )
    
    # Calcula MDE atual
    z_alpha = stats.norm.ppf(0.975)
    z_poder = stats.norm.ppf(poder_atual)
    variancia = std ** 2
    erro_padrao = np.sqrt(variancia/n_target + variancia/n_control)
    mde_atual = (z_alpha + z_poder) * erro_padrao
    
    print(f"📊 MDE ATUAL:")
    print(f"• Poder: {poder_atual:.1%}")
    print(f"• MDE: R$ {mde_atual:.2f}")
    print(f"• Diferença observada: R$ {media_target - media_control:.2f}")
    
    return mde_atual

# Uso
mde = calcular_mde_completo(df_publico_mes)

In [ ]:
# "Qual o poder para detectar R$ 1,00?"
mde_desejado = 1.00
effect_size_desejado = mde_desejado / std  # std = 30.22

poder_para_1_real = analysis.solve_power(
    effect_size=effect_size_desejado,
    nobs1=n_target, 
    alpha=0.05,
    ratio=n_control/n_target
)
poder_para_1_real

In [ ]:
from statsmodels.stats.power import TTestIndPower

# Use sua amostra atual
n_target = len(df_publico_mes[df_publico_mes['is_target'] == 'target'])
n_control = len(df_publico_mes[df_publico_mes['is_target'] == 'control'])
std = df_publico_mes['amount_pos_desc'].std()

# Calcule o MDE para 80% de poder
analysis = TTestIndPower()
effect_size_80poder = analysis.solve_power(
    effect_size=None,  # ← Ele calcula este valor
    nobs1=n_target,
    power=0.8,         # 80% de poder
    alpha=0.05,
    ratio=n_control/n_target
)

mde_80poder = effect_size_80poder * std
print(f"MDE com 80% de poder: R$ {mde_80poder:.2f}")

In [ ]:
# Mostra como o poder varia com o tamanho do efeito
from statsmodels.stats.power import TTestIndPower
import numpy as np

analysis = TTestIndPower()
analysis.plot_power(dep_var='effect_size', nobs=[n_target], effect_size=np.arange(0.01, 0.2, 0.01))
plt.title('Curva de Poder vs Tamanho do Efeito')
plt.show()

In [ ]:
# 📊 CÁLCULO COMPLETO DE VIABILIDADE FINANCEIRA

# Dados do seu teste
custo_desconto = 99794.61
receita_liquida_target = 796648.08  
receita_control = 606084.71

# Cálculos
receita_adicional_real = receita_liquida_target - receita_control
lucro_estrategia = receita_adicional_real - custo_desconto
roi = (lucro_estrategia / custo_desconto) * 100

print("📊 VIABILIDADE FINANCEIRA DA ESTRATÉGIA")
print(f"1. Custo total do desconto: R$ {custo_desconto:,.2f}")
print(f"2. Receita adicional REAL: R$ {receita_adicional_real:,.2f}")
print(f"3. Lucro líquido da estratégia: R$ {lucro_estrategia:,.2f}")
print(f"4. ROI: {roi:.1f}% (cada R$ 1 gerou R$ {roi/100:.2f})")

"""
📊 RELATÓRIO FINAL - TESTE A/B COM 20% DESCONTO

🎯 RESULTADOS ESTATÍSTICOS

CONVERSÃO (✓ ALTAMENTE SIGNIFICATIVO)
• Control: 24.12% dos clientes voltaram
• Target: 27.47% dos clientes voltaram  
• Lift: +13.9% (+3.404 clientes adicionais)
• Significância: p-value ≈ 0 (resultado REAL)

VALOR POR CLIENTE (⚠️ LIMITAÇÃO ESTATÍSTICA)
• Diferença: R$ 0,36 (target: R$ 20,58 vs control: R$ 20,22)
• Poder estatístico: 33.4% (BAIXO)
• MDE atual: R$ 0,37 (detecta apenas diferenças grandes)
• MDE para 80% poder: R$ 0,67 (necessário para confiança)

💰 VIABILIDADE FINANCEIRA (✓ POSITIVA)

RETORNO DO INVESTIMENTO
• Custo desconto: R$ 99.794,61
• Receita adicional: R$ 190.563,37
• Lucro líquido: R$ 90.768,76
• ROI: 91% (cada R$ 1 gerou R$ 0,91)

🎯 CONCLUSÕES E RECOMENDAÇÕES

✅ ESTRATÉGIA FUNCIONA PARA:
• Aumentar conversão para segunda compra
• Gerar retorno financeiro positivo
• Atrair mais clientes de volta

⚠️ PONTOS DE OTIMIZAÇÃO:
• Poder estatístico baixo para diferenças pequenas
• Desconto de 20% pode ser reduzido
• Testar janelas maiores que 5 dias

💡 PRÓXIMOS PASSOS SUGERIDOS:
1. Testar com 10-15% de desconto
2. Estender janela para 7-10 dias
3. Monitorar ticket médio dos convertidos
"""

Clientes com um unico pedido

In [ ]:
x = 1 # Exemplo: clientes com 2 pedidos

df_clientes_x_pedidos = (
    df[df['order_created_month'] == 12]
    .groupby(['customer_id', 'order_created_month'])
    .agg(
        quantidade_pedidos=('unique_order_hash', 'count')
    )
    .reset_index()
    .query(f'quantidade_pedidos == {x}')
)


In [ ]:
df_=df[['customer_id','order_created_month','merchant_id','order_created_at','days_until_next_order','rank_month','order_total_amount','amount_pct_change_from_previous','amount_diff_from_previous']]

In [ ]:
# 1. Valor das 2ªs ordens COM cupom
valor_segundas_ordens = df[df['rank_month'] == 2]['order_total_amount'].sum()

# 2. Custo total do cupom
custo_cupom = df[df['rank_month'] == 2]['%desconto'].sum()

# 3. Receita líquida das 2ªs ordens
receita_liquida = valor_segundas_ordens - custo_cupom

print("💰 IMPACTO DO CUPOM NA 2ª ORDEM:")
print(f"• Valor bruto das 2ªs ordens: R$ {valor_segundas_ordens:,.2f}")
print(f"• Custo do cupom: R$ {custo_cupom:,.2f}")
print(f"• Receita líquida: R$ {receita_liquida:,.2f}")
print(f"• % do custo sobre a receita: {(custo_cupom/valor_segundas_ordens*100):.1f}%")

In [ ]:


# Mostrar amostra
df[['customer_id', 'order_created_month','order_created_month' ,'rank_month', 'order_total_amount', '%desconto']].head(10)

In [ ]:
# Filtrar a base df apenas com os clientes que têm X pedidos
df_filtrado = df_[
    (df_['customer_id'].isin(df_clientes_x_pedidos['customer_id'])) #& 
    #(df['order_created_month'] == 12)
]
#df_filtrado.sort_values('amount_pct_change_from_previous')

In [ ]:
df_filtrado.head()

In [ ]:
from lifelines.statistics import logrank_test

# Supondo que você tem analysis_df com:
# - customer_id (identificador único)
# - grupo_ab ('A' ou 'B') 
# - tempo_ate_segunda (dias)
# - fez_segunda_ordem (True/False)

# Separar os grupos
grupo_a = clientes_unicos_primeiro_pedido[clientes_unicos_primeiro_pedido['is_target'] == 'target']
grupo_b = clientes_unicos_primeiro_pedido[clientes_unicos_primeiro_pedido['is_target'] == 'contol']

print(f"Grupo A: {len(grupo_a)} clientes")
print(f"Grupo B: {len(grupo_b)} clientes")

# Teste de Log-Rank
results = logrank_test(
    grupo_a['tempo_ate_segunda'], 
    grupo_b['tempo_ate_segunda'],
    event_observed_A=grupo_a['fez_segunda_ordem'],
    event_observed_B=grupo_b['fez_segunda_ordem']
)

print("\n=== TESTE DE LOG-RANK ===")
print(f"p-value: {results.p_value:.4f}")
print(f"Significativo (p < 0.05): {results.p_value < 0.05}")

if results.p_value < 0.05:
    print("✅ Há diferença significativa entre os grupos!")
else:
    print("❌ Não há diferença significativa entre os grupos")

In [ ]:
# Só precisa das contagens totais
from scipy.stats import chi2_contingency

# Criar tabela manualmente se não tiver o dataset
tabela = np.array([
    [len(grupo_a[grupo_a['fez_segunda_ordem'] == False]),  # Grupo A não converteu
     len(grupo_a[grupo_a['fez_segunda_ordem'] == True])],   # Grupo A converteu
    
    [len(grupo_b[grupo_b['fez_segunda_ordem'] == False]),  # Grupo B não converteu  
     len(grupo_b[grupo_b['fez_segunda_ordem'] == True])]    # Grupo B converteu
])

chi2, p_value, dof, expected = chi2_contingency(tabela)

In [ ]:
conversao_ab

In [ ]:
from scipy.stats import chi2_contingency

# Tabela de contingência
tabela = pd.crosstab(analysis_df['grupo_ab'], analysis_df['fez_segunda_ordem'])
print("=== TABELA DE CONTINGÊNCIA ===")
print(tabela)

# Teste Chi-Quadrado
chi2, p_value, dof, expected = chi2_contingency(tabela)
print(f"\n=== TESTE CHI-QUADRADO ===")
print(f"Chi2: {chi2:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Significativo (p < 0.05): {p_value < 0.05}")

In [ ]:
df.head()

In [ ]:
from datetime import datetime
data_inicio = datetime(2024, 1, 1)
data_inicio

In [ ]:
clientes_ativos_janeiro = df[df['order_created_month'] == 1]['customer_id'].unique()
df_clientes_jan = df[df['customer_id'].isin(clientes_ativos_janeiro)].copy()

In [ ]:
df=df[df['order_created_month'] == 1]

In [ ]:
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt

# Clientes ativos em janeiro de 2019 (fizeram pelo menos 1 pedido em jan/2019)
clientes_ativos_janeiro = df[
    (df['order_created_month'] == 1) & 
    (df['order_created_at'].dt.year == 2019)
]['customer_id'].unique()

print(f"Total de clientes ativos em janeiro de 2019: {len(clientes_ativos_janeiro)}")

# Filtrar TODOS os pedidos desses clientes
df_clientes_jan = df[df['customer_id'].isin(clientes_ativos_janeiro)].copy()

# Preparar dados para análise de sobrevivência CORRETA
analysis_data = []

for customer_id in clientes_ativos_janeiro:
    customer_orders = df_clientes_jan[df_clientes_jan['customer_id'] == customer_id].sort_values('order_created_at')
    
    # Primeira ordem do cliente em janeiro de 2019
    ordens_janeiro_2019 = customer_orders[
        (customer_orders['order_created_month'] == 1) & 
        (customer_orders['order_created_at'].dt.year == 2019)
    ]
    
    if len(ordens_janeiro_2019) >= 1:
        primeira_ordem_jan = ordens_janeiro_2019.iloc[0]
        data_primeira_ordem = primeira_ordem_jan['order_created_at']
        
        # Verificar se fez segunda ordem (em qualquer mês/ano)
        fez_segunda_ordem = len(customer_orders) >= 2
        
        if fez_segunda_ordem:
            segunda_ordem = customer_orders.iloc[1]
            data_segunda_ordem = segunda_ordem['order_created_at']
            
            # CORREÇÃO: Calcular tempo ENTRE 1ª e 2ª ordem
            tempo_ate_segunda = (data_segunda_ordem - data_primeira_ordem).days
            
            # Garantir que não seja negativo
            if tempo_ate_segunda < 0:
                print(f"AVISO: Tempo negativo para cliente {customer_id}")
                tempo_ate_segunda = 0
        else:
            # Cliente não fez segunda ordem - censurar após período razoável
            tempo_ate_segunda = 90  # Censurar em 90 dias
            fez_segunda_ordem = False
        
        analysis_data.append({
            'customer_id': customer_id,
            'grupo_ab': primeira_ordem_jan.get('grupo_ab', 'unknown'),
            'tempo_ate_segunda': tempo_ate_segunda,
            'fez_segunda_ordem': fez_segunda_ordem,
            'data_primeira_ordem': data_primeira_ordem
        })

# Criar DataFrame de análise
analysis_df = pd.DataFrame(analysis_data)

print(f"Clientes na análise: {len(analysis_df)}")
print(f"Clientes que fizeram 2ª ordem: {analysis_df['fez_segunda_ordem'].sum()}")
print(f"Taxa de conversão: {analysis_df['fez_segunda_ordem'].mean():.2%}")

# Verificar se há tempos negativos
if (analysis_df['tempo_ate_segunda'] < 0).any():
    print("⚠️  AVISO: Ainda há tempos negativos!")
    print(analysis_df[analysis_df['tempo_ate_segunda'] < 0])

# Aplicar Kaplan-Meier
kmf = KaplanMeierFitter()
kmf.fit(
    durations=analysis_df['tempo_ate_segunda'],
    event_observed=analysis_df['fez_segunda_ordem']
)

# Plotar
plt.figure(figsize=(12, 8))
kmf.plot_survival_function()
plt.title('Tempo entre 1ª e 2ª Ordem\n(Clientes Janeiro 2019)')
plt.ylabel('Probabilidade de Ainda Não Ter Feito 2ª Ordem')
plt.xlabel('Dias desde a Primeira Ordem')
plt.grid(True)
plt.show()

# Estatísticas
print(f"\n--- Estatísticas ---")
print(f"Mediana de tempo até 2ª ordem: {kmf.median_survival_time_} dias")
for dias in [1, 3, 7, 15, 30]:
    conversao = (1 - kmf.predict(dias)) * 100
    print(f"Conversão em {dias} dias: {conversao:.1f}%")

In [ ]:
second_orders = df_f[df_f['rank_month'] == 2]

In [ ]:
df_f

In [ ]:
df_g = (
        second_orders.groupby(['order_created_month','days_since_first_order_month'])
          .agg(
              total_clientes=('customer_id', 'nunique'),
              total_pedidos=('order_id', 'count'),
              total_ordem=('order_total_amount', 'sum')
          )
          .reset_index()
          .assign(
              perc_clientes=lambda d: d.groupby('order_created_month')['total_clientes']
                                         .transform(lambda x: x / x.sum() * 100),
              perc_pedidos=lambda d: d.groupby('order_created_month')['total_pedidos']
                                        .transform(lambda x: x / x.sum() * 100),
              perc_amount=lambda d: d.groupby('order_created_month')['total_ordem']
                                         .transform(lambda x: x / x.sum() * 100)
          )
          .round(2)
          .sort_values(['order_created_month', 'days_since_first_order_month'], ascending=[True, True])
          .reset_index(drop=True)
    )
df_g[df_g['order_created_month']==12]
#df_g[df_g['order_created_month']==12]['total_ordem'].sum()


In [ ]:
df.head()

In [ ]:
gerar_stats(df,['day']).head()

In [ ]:
import streamlit as st

df.to_parquet(BASE_PATH / "gold" / "df_app.parquet", index=False)
estado = st.selectbox("Escolha o estado", df["weekday"].unique())
#mes = st.selectbox("Escolha o mês", df["month"].unique())

df_filtrado = df[(df["weekday"] == estado) #& (df["month"] == mes)
                 ]
st.dataframe(df_filtrado)


In [ ]:
df_pub_u=process_orders_pandas(df_pub_u)

In [ ]:
df_pub_un = df_pub_u[["customer_id","is_target", "order_created_month", "num_pedidos_mes", "num_pedidos_hist",'total_amount_mes','ticket_medio']].drop_duplicates().reset_index(drop=True)
df_pub_un['pedidos_sum'] = np.where(
        df_pub_un['num_pedidos_mes'] > 10, '10+',
        df_pub_un['num_pedidos_mes'].astype(str)
    )
df_pub_un.head()

In [ ]:
df_pub_un = df[["customer_id","is_target", "order_created_month", "num_pedidos_mes", "num_pedidos_hist",'total_amount_mes','ticket_medio']].drop_duplicates().reset_index(drop=True)
df_pub_un['pedidos_sum'] = np.where(
        df_pub_un['num_pedidos_mes'] > 10, '10+',
        df_pub_un['num_pedidos_mes'].astype(str)
    )
df_pub_un.head()